In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.2/488.2 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 90.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 114.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.0/441.0 kB 48.1 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.3.3
    Uninstalling flatbuffers-23.3.3:
      Successfully uninstalled flatbuffers-23.3.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2.14.0-dev20230528


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2023-05-28 16:49:28--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2023-05-28 16:49:29 (13.1 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2023-05-28 16:49:29--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [3]:
# Load train and test sets using pandas
train_df = pd.read_csv(train_file_path, sep='\t', header=None)
test_df = pd.read_csv(test_file_path, sep='\t', header=None)

train_df.head()

,0,1
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [4]:
# Split train and test data into text (features) and labels
X_train, y_train = train_df[1].values, train_df[0].values
X_test, y_test = test_df[1].values, test_df[0].values

In [5]:
# Convert label strings to 0s and 1s
y_train, y_test = np.where(y_train == "ham", 0, 1), np.where(y_test == "ham", 0, 1)


In [6]:
# Tokenize the text data using the Tokenizer class from keras
tokenizer = keras.preprocessing.text.Tokenizer(num_words=1000, lower=True, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

In [7]:
# Convert text data to sequences of integer indices
X_train_seqs = tokenizer.texts_to_sequences(X_train)
X_test_seqs = tokenizer.texts_to_sequences(X_test)

In [8]:
# Pad sequences to a fixed length of 100
X_train_padded = keras.preprocessing.sequence.pad_sequences(X_train_seqs, maxlen=50, padding='post', truncating='post')
X_test_padded = keras.preprocessing.sequence.pad_sequences(X_test_seqs, maxlen=50, padding='post', truncating='post')

In [9]:
# Create the model
max_words = 1000
max_len = 50

i = tf.keras.layers.Input(shape=[max_len])
x = tf.keras.layers.Embedding(max_words, 50, input_length=max_len)(i)
x = tf.keras.layers.LSTM(64)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(inputs=i, outputs=x)

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Model Summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50)]              0         
                                                                 
 embedding (Embedding)       (None, 50, 50)            50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 dense (Dense)               (None, 256)               16640     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 96337 (376.32 KB)
Trainable params: 96337 (376.

In [10]:
# Train the model with early stopping
early_stop = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)
history = model.fit(X_train_padded, y_train, epochs=10, batch_size=128, validation_data=(X_test_padded, y_test), callbacks=[early_stop])

Epoch 1/10
33/33 [==============================] - 20s 363ms/step - loss: 0.4748 - accuracy: 0.8545 - val_loss: 0.3623 - val_accuracy: 0.8657
Epoch 2/10
33/33 [==============================] - 3s 93ms/step - loss: 0.2122 - accuracy: 0.9165 - val_loss: 0.1114 - val_accuracy: 0.9641
Epoch 3/10
33/33 [==============================] - 3s 74ms/step - loss: 0.0736 - accuracy: 0.9828 - val_loss: 0.0573 - val_accuracy: 0.9813
Epoch 4/10
33/33 [==============================] - 1s 40ms/step - loss: 0.0451 - accuracy: 0.9861 - val_loss: 0.0474 - val_accuracy: 0.9864
Epoch 5/10
33/33 [==============================] - 1s 42ms/step - loss: 0.0311 - accuracy: 0.9928 - val_loss: 0.0563 - val_accuracy: 0.9842
Epoch 6/10
33/33 [==============================] - 2s 55ms/step - loss: 0.0215 - accuracy: 0.9950 - val_loss: 0.0908 - val_accuracy: 0.9813


In [11]:
# Make predictions
y_pred_proba = model.predict(X_test_padded)
y_pred = np.where(y_pred_proba > 0.5, 1, 0)

44/44 [==============================] - 1s 3ms/step


In [12]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test)
print("Test set accuracy:", accuracy)

44/44 [==============================] - 0s 5ms/step - loss: 0.0474 - accuracy: 0.9864
Test set accuracy: 0.9863505959510803


In [13]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

    # Preprocess the[pred_text (note: this assumes the tokenizer and padding have already been done on the full dataset)
    message_seq = tokenizer.texts_to_sequences([pred_text])
    message_padded = keras.preprocessing.sequence.pad_sequences(message_seq, maxlen=100, padding='post', truncating='post')
    # Make a prediction
    proba = model.predict(message_padded)[0][0]
    if proba > 0.6:
        label = "spam"
    else:
        label = "ham"
    return [proba, label]


pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 499ms/step
[0.0014784877, 'ham']


In [14]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 17ms/step
You passed the challenge. Great job!
